<a href="https://colab.research.google.com/github/L-Gaysina/HW4_Biostat/blob/main/Cozyduke_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Сервис автоматического анализа текста двух документов и присвоения статуса соответсвия
<hr>


Над проектом работает:
Команда **'COZYDUKE'.**
0. Кирсанов Вадим Вадимович
1. Казаченко Екатерина Александровна
2. Кириленков Кирилл Владимирович
3. Гайсина Лиана Ильдаровна
4. Федеряев Клим Александрович
5. Бадретдинова Рушания Ринатовна



In [1]:
# @title Дополнительные модули
# Установка дополнительных модулей
%pip install python-docx
%pip install POT
%pip install torch

# Импорт библиотек и их модулей
import re
from io import BytesIO

import docx
import gensim.downloader as api
import numpy as np
import pandas as pd
import requests
import torch
from gensim.models import KeyedVectors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity


# 1. Подготовка данных
* **Извлечение данных:** Данные были извлечены из файлов в формате DOCX с использованием библиотеки python-docx.

* **Предобработка текста**: Произведена предобработка текстовых данных, включая удаление знаков пунктуации, приведение всех букв к нижнему регистру и удаление лишних пробелов. Эти шаги были выполнены с помощью регулярных выражений и методов обработки строк в Python.

* **Создание датафреймов:** Составлено 2 датафрейма для каждого DOCX-документа, каждый из которых хранит весь текст из одного документа разбитого по абзацам



In [2]:
# Загрузка документов
# @title Для корректного выполнения кода вы можете заменить путь к файлу на свой в переменных docx_url и docx_url_req
def download_docx_from_url(url):
    response = requests.get(url)
    response.raise_for_status()
    return docx.Document(BytesIO(response.content))

# URL документа с характеристиками и требованиями
docx_url = 'https://docs.google.com/document/d/1iJPrBdchU1ZeXv-oW9cokJP1GN72WDGE/export?format=docx'
docx_url_req = 'https://docs.google.com/document/d/1TNUi3yUs_GFEhXOOTJE05ejvOXYdkQLe/export?format=docx'

# Загрузка документа Word
doc = download_docx_from_url(docx_url)
doc_req = download_docx_from_url(docx_url_req)

In [3]:
# @title Создание датафреймов

def parse_paragraphs_to_dataframe(doc):
    all_paragraphs = []
    current_header = "Без заголовка"  # Начальный заголовок для текста без явного разделения

    for para in doc.paragraphs:
        if para.text.strip():  # Проверка на непустой текст
            # Ищем пункты или заголовки, определяем по форматированию или содержимому
            if para.style.name.startswith('Heading') or (para.text.split()[0].replace('.', '').isdigit() and len(para.text.split()) > 1):
                current_header = para.text.split(maxsplit=1)[0] if para.text.split()[0].replace('.', '').isdigit() else para.text
            all_paragraphs.append((current_header, para.text))

    return pd.DataFrame(all_paragraphs, columns=['Заголовок', 'Текст параграфа'])

def add_section_numbers(df, text_column):
    # Компилируем регулярное выражение для поиска номеров пунктов
    pattern = re.compile(r'^(\d+(\.\d+)*)\s')

    # Функция для извлечения номера пункта из строки
    def extract_number(text):
        match = pattern.match(text)
        return match.group(1) if match else None

    # Применяем функцию к каждому элементу в столбце текста и создаем новую колонку
    df['Номер пункта'] = df[text_column].apply(extract_number)
    return df

# Парсим абзацы в DataFrame
df_all_text = parse_paragraphs_to_dataframe(doc)
df_all_text_req =  parse_paragraphs_to_dataframe(doc_req)

# Создаем колонку с номерами пунктов
df_all_text_req = add_section_numbers(df_all_text_req, 'Текст параграфа')
df_all_text = add_section_numbers(df_all_text, 'Текст параграфа')
df_all_text.head()


,Заголовок,Текст параграфа,Номер пункта
0,Без заголовка,СОДЕРЖАНИЕ,None
1,Без заголовка,Настоящие технические условия распространяются...,None
2,Без заголовка,Агрегаты не предназначены для перекачивания кр...,None
3,Без заголовка,Допускается перекачивание жидкостей с температ...,None
4,Без заголовка,Агрегаты изготавливаются в климатическом испол...,None


In [4]:
# @title Обработка текста
def preprocess_text(df, text_column):
    df[text_column] = df[text_column].apply(lambda x: re.sub(r'[^\w\s]', '', x.lower()).replace('\n', ' ').strip())
    df[text_column] = df[text_column].apply(lambda x: re.sub(r'\s+', ' ', x))
    return df

def remove_short_requirements(df, column_name='текст требования'):
    """
    Удаляет строки из DataFrame, если количество слов в указанной колонке меньше 5.

    Параметры:
    df (pd.DataFrame): исходный DataFrame, из которого требуется удалить строки.
    column_name (str): имя колонки, в которой проверяется количество слов.

    Возвращает:
    pd.DataFrame: DataFrame после удаления строк.
    """
    # Фильтрация DataFrame: оставляем только те строки, где количество слов в column_name >= 5
    filtered_df = df[df[column_name].apply(lambda x: len(str(x).split()) >= 5)]

    return filtered_df

# Предварительная обработка текста в DataFrame
df_all_text = preprocess_text(df_all_text, 'Текст параграфа')
df_all_text_req = preprocess_text(df_all_text_req, 'Текст параграфа')

# Удаляем строки содержащие менее 5 слов
df_all_text_req = remove_short_requirements(df_all_text_req, 'Текст параграфа')
df_all_text = remove_short_requirements(df_all_text, 'Текст параграфа')

# Обновляем индекссацию после удаления
df_all_text_req = df_all_text_req.reset_index(drop=True)
df_all_text = df_all_text.reset_index(drop=True)

# Вывод результата
df_all_text.head(40)

,Заголовок,Текст параграфа,Номер пункта
0,Без заголовка,настоящие технические условия распространяются...,None
1,Без заголовка,агрегаты не предназначены для перекачивания кр...,None
2,Без заголовка,допускается перекачивание жидкостей с температ...,None
3,Без заголовка,агрегаты изготавливаются в климатическом испол...,None
4,Без заголовка,в условиях умеренного климата у2 45 40 с,None
5,Без заголовка,в условиях умеренного и холодного климата ухл1...,None
6,Без заголовка,в условиях холодного климата хл1хл2 6040 с,None
7,Без заголовка,в условиях умереннохолодного и тропического мо...,None
8,Без заголовка,допустимый диапазон температуры окружающей сре...,None
9,Без заголовка,агрегаты могут поставляться в общепромышленном...,None


In [5]:
# Вывод результата
df_all_text_req.head(40)

,Заголовок,Текст параграфа,Номер пункта
0,Введение,использование настоящего стандарта в конкретны...,None
1,Введение,настоящий стандарт является модифицированным п...,None
2,Введение,дополнительные положения и требования а также ...,None
3,Введение,знак в начале параграфа или его раздела указыв...,None
4,Введение,из соображений удобства и в информационных цел...,None
5,Область применения,настоящий стандарт устанавливает требования к ...,None
6,Область применения,настоящий стандарт применяется к одноступенчат...,None
7,Область применения,п р и м е ч а н и е для распространения действ...,None
8,Область применения,опыт промышленной эксплуатации герметичных нас...,None
9,Область применения,давление на выходе 1900 кпа 275 psig,None


# 2. Векторизация текста и расчёт сходства
* **Векторизация текста**: Используем TF-IDF векторизацию для преобразования текстов требований и параграфов в числовые векторы и подготовки данных к анализу сходства.

* **Расчет косинусного сходства**: Вычисляем косинусное сходство между векторами параграфов и требований для определения степени соответствия между ними.

In [5]:
# @title Векторизация текста и расчёт косинусного сходства

# Подготовка текста: соберем все тексты параграфов в список
texts = df_all_text['Текст параграфа'].tolist()

# Подготовка списка текстов требований из предыдущей предобработки (словаря требований)
requirement_texts = df_all_text_req['Текст параграфа'].tolist()

# Объединение текстов требований и текстов параграфов для обучения одного TF-IDF векторизатора
combined_texts = texts + requirement_texts

# Создание и обучение TF-IDF векторизатора на объединенном корпусе
combined_tfidf_vectorizer = TfidfVectorizer()
combined_tfidf_matrix = combined_tfidf_vectorizer.fit_transform(combined_texts)

# Разделение матрицы TF-IDF на часть параграфов и часть требований
tfidf_paragraphs = combined_tfidf_matrix[:len(texts), :]
tfidf_requirements = combined_tfidf_matrix[len(texts):, :]

# Расчет косинусного сходства между параграфами и требованиями
cosine_similarities = cosine_similarity(tfidf_paragraphs, tfidf_requirements)

# Вывод формы матрицы сходств для проверки
cosine_similarities.shape


(179, 1676)

In [6]:
'''

# Инициализация токенизатора и модели
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-large')

# Функция для average pooling
def average_pool(last_hidden_states, attention_mask):
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

# Объединение текстов параграфов и требований
combined_texts = texts + requirement_texts

# Токенизация текстов
inputs = tokenizer(combined_texts, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Перенос данных на GPU, если доступно
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
inputs = {key: value.to(device) for key, value in inputs.items()}

# Получение эмбеддингов от модели
with torch.no_grad():
    outputs = model(**inputs)
    embeddings = average_pool(outputs.last_hidden_state, inputs['attention_mask'])

# Нормализация эмбеддингов
embeddings = F.normalize(embeddings, p=2, dim=1)

# Разделение эмбеддингов на параграфы и требования
paragraph_embeddings = embeddings[:len(texts)]
requirement_embeddings = embeddings[len(texts):]

# Расчет косинусного сходства
cosine_similarities = cosine_similarity(paragraph_embeddings.cpu().numpy(), requirement_embeddings.cpu().numpy())

# Вывод формы матрицы сходств для проверки
print(cosine_similarities.shape)
'''

'\n\n# Инициализация токенизатора и модели\ntokenizer = AutoTokenizer.from_pretrained(\'intfloat/multilingual-e5-large\')\nmodel = AutoModel.from_pretrained(\'intfloat/multilingual-e5-large\')\n\n# Функция для average pooling\ndef average_pool(last_hidden_states, attention_mask):\n    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)\n    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]\n\n# Объединение текстов параграфов и требований\ncombined_texts = texts + requirement_texts\n\n# Токенизация текстов\ninputs = tokenizer(combined_texts, return_tensors="pt", padding=True, truncation=True, max_length=512)\n\n# Перенос данных на GPU, если доступно\ndevice = torch.device("cuda" if torch.cuda.is_available() else "cpu")\nmodel = model.to(device)\ninputs = {key: value.to(device) for key, value in inputs.items()}\n\n# Получение эмбеддингов от модели\nwith torch.no_grad():\n    outputs = model(**inputs)\n    embeddings = average_pool(outpu

#3. Определение наилучших совпадений
* **Анализ сходства:** Для каждого требования определяем абзац из второго файла с максимальным уровнем сходства и выявляем наиболее релевантные соответствия.

* **Создание итогового отчета:** Результаты анализа собираем в датафрейме, который включает информацию о требованиях, соответствующих параграфов  и степени их сходства и выгружаем отчет в excel-файле.

In [7]:
# Инициализация пустого DataFrame для наилучших совпадений
best_match_df = pd.DataFrame(columns=['Пункт требования', 'Текст требования', 'Заголовок параграфа', 'Пункт параграфа', 'Текст параграфа', 'Сходство'])

# Сбор всех строк для добавления
rows_to_add = []
for req_index, req_row in df_all_text_req.iterrows():
    paragraph_index = np.argmax(cosine_similarities[:, req_index])
    max_similarity = cosine_similarities[paragraph_index, req_index]
    paragraph_data = df_all_text.loc[paragraph_index]
    paragraph_data_req = df_all_text_req.loc[paragraph_index]
    # Создание временного DataFrame для текущей строки
    temp_df = pd.DataFrame({
        'Заголовок требования': [req_row['Заголовок']],
        'Номер пункта требования': [req_row['Номер пункта']],
        'Текст требования': [req_row['Текст параграфа']],
        'Заголовок параграфа': [paragraph_data['Заголовок']],
        'Номер пункта параграфа': [paragraph_data['Номер пункта']],  # Предполагая, что 'Номер пункта' есть в df_all_text
        'Текст параграфа': [paragraph_data['Текст параграфа']],
        'Сходство': [max_similarity]
    })
    rows_to_add.append(temp_df)

# Добавление всех собранных строк в основной DataFrame
best_match_df = pd.concat(rows_to_add, ignore_index=True)

# Вывод первых нескольких строк для проверки
best_match_df.head(40)


,Заголовок требования,Номер пункта требования,Текст требования,Заголовок параграфа,Номер пункта параграфа,Текст параграфа,Сходство
0,Введение,None,использование настоящего стандарта в конкретны...,Требования надежности,None,указанные ресурсы и сроки службы действительны...,0.080409
1,Введение,None,настоящий стандарт является модифицированным п...,Без заголовка,None,по согласованию заказчика разработчика и завод...,0.116601
2,Введение,None,дополнительные положения и требования а также ...,Маркировка,None,табличка агрегатов электронасосных устанавлива...,0.047446
3,Введение,None,знак в начале параграфа или его раздела указыв...,Методы контроля,None,материалы проверка сертификатов или проведение...,0.090388
4,Введение,None,из соображений удобства и в информационных цел...,Методы контроля,None,в процессе изготовления и приемки деталей и сб...,0.179324
5,Область применения,None,настоящий стандарт устанавливает требования к ...,Без заголовка,None,эксплуатация насоса с наличием паровой газовой...,0.076962
6,Область применения,None,настоящий стандарт применяется к одноступенчат...,7.14,None,контроль температуры герметизирующего экрана д...,0.078046
7,Область применения,None,п р и м е ч а н и е для распространения действ...,7.14,None,714 для предотвращения отказов и аварий насосы...,0.088502
8,Область применения,None,опыт промышленной эксплуатации герметичных нас...,Требования безопасности,None,при эксплуатации агрегата запрещается устранят...,0.107133
9,Область применения,None,давление на выходе 1900 кпа 275 psig,Основные параметры и характеристики,None,максимальное допускаемое давление на входе в н...,0.078796


In [8]:
# Функция для категоризации сходства
def categorize_similarity(similarity):
    if similarity >= 0.1:
        return "Соответствует"
    elif similarity >= 0.07:
        return "Частично соответствует"
    else:
        return "Не соответствует"

# Применение функции к колонке 'Сходство'
best_match_df['Степень соответствия'] = best_match_df['Сходство'].apply(categorize_similarity)
best_match_df

,Заголовок требования,Номер пункта требования,Текст требования,Заголовок параграфа,Номер пункта параграфа,Текст параграфа,Сходство,Степень соответствия
0,Введение,None,использование настоящего стандарта в конкретны...,Требования надежности,None,указанные ресурсы и сроки службы действительны...,0.080409,Частично соответствует
1,Введение,None,настоящий стандарт является модифицированным п...,Без заголовка,None,по согласованию заказчика разработчика и завод...,0.116601,Соответствует
2,Введение,None,дополнительные положения и требования а также ...,Маркировка,None,табличка агрегатов электронасосных устанавлива...,0.047446,Не соответствует
3,Введение,None,знак в начале параграфа или его раздела указыв...,Методы контроля,None,материалы проверка сертификатов или проведение...,0.090388,Частично соответствует
4,Введение,None,из соображений удобства и в информационных цел...,Методы контроля,None,в процессе изготовления и приемки деталей и сб...,0.179324,Соответствует
...,...,...,...,...,...,...,...,...
1671,Приложение S,None,сведения о соответствии межгосударственных ста...,"Требования к материалам, покупным изделиям и и...",None,все материалы поступающие в производство для и...,0.056442,Не соответствует
1672,Библиография,None,раздел 1 иностранные стандарты и нормативные д...,"Требования к материалам, покупным изделиям и и...",None,все материалы поступающие в производство для и...,0.102429,Соответствует
1673,Библиография,None,американская ассоциация инженеровмехаников рук...,Комплектность,None,паспорт и руководство по эксплуатации 1 комплект,0.197749,Соответствует
1674,РАЗДЕЛ 2. Иностранные стандарты на материалы,None,раздел 2 иностранные стандарты на материалы,Правила приемки,None,материалы покупные и комплектующие изделия дол...,0.089101,Частично соответствует


In [12]:
# @title Скачиваем итоговый файл
# Путь для сохранения файла Excel
file_path = 'output.xlsx'

# Сохранение DataFrame в файл Excel
best_match_df.to_excel(file_path, index=False, engine='openpyxl')

#4. Изучение альтернативных способов определения сходства

* **Использование модели BERT**: Для начала, был загружен предобученный токенизатор и модель BERT (bert-base-uncased) из библиотеки Hugging Face. Эта модель является одной из базовых версий BERT, предназначенной для работы с текстами.

* **Работа с текстами**
Списки текстов из параграфов и требований были подготовлены для обработки. Тексты были извлечены из датафрейма best_match_df, который содержал информацию о наилучшем соответствии между параграфами и требованиями.Для каждого списка текстов были получены соответствующие эмбеддинги. С использованием функции косинусного сходства было рассчитано соответствие между параграфами и требованиями.

* Однако полученные результаты оказались чересчур оптимистичными и  малоинформативными.

In [ ]:
# Загрузка предобученного токенизатора и модели BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased', return_dict=True)

def get_bert_embeddings(texts):
    """Функция для получения BERT эмбеддингов."""
    encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt', max_length=512)
    with torch.no_grad():
        outputs = model(**encoded_input)
    # Используем [CLS] токен для представления всего текста
    embeddings = outputs.last_hidden_state[:, 0, :].numpy()
    return embeddings

# Подготовка текстов
texts = best_match_df['Текст параграфа'].tolist()
requirements = best_match_df['Текст требования'].tolist()

# Получение эмбеддингов
paragraph_embeddings = get_bert_embeddings(texts)
requirement_embeddings = get_bert_embeddings(requirements)

# Расчет косинусного сходства
cosine_sim = cosine_similarity(paragraph_embeddings, requirement_embeddings)

# Сохранение сходства в DataFrame
for idx in range(len(best_match_df)):
    best_match_df.at[idx, 'Сходство BERT'] = cosine_sim[idx, idx]  # диагональные элементы, если порядок соответствует

# Вывод первых строк DataFrame для проверки
best_match_df.head(20)


tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

* **Загрузка модели Word2Vec:** Использовалась предобученная модель Word2Vec ('word2vec-google-news-300') из коллекции Google News, загруженная через API gensim.

* **Подготовка модели:** После загрузки убедились, что модель доступна как объект KeyedVectors для использования функции wmdistance, которая позволяет вычислять расстояние по методу Word Mover's Distance (WMD) между двумя текстами.

* **Функция расчёта WMD:** Реализована функция calculate_wmd, которая принимает два текста, преобразует их в нижний регистр, разбивает на слова и вычисляет WMD. Этот метод основан на минимальном расстоянии, необходимом для преобразования всех слов в одном документе в слова другого документа, учитывая их семантическую близость.

* **Применение WMD к данным:** Для каждой пары текстов (требования и соответствующий параграф) в DataFrame best_match_df было вычислено WMD, что позволило оценить их семантическую близость. Результаты сохранены в столбец WMD_Similarity данного DataFrame.

In [ ]:
# Загрузка предобученной модели Word2Vec
model = api.load('word2vec-google-news-300')

# Обеспечение, что модель полностью загружена как KeyedVectors для доступа к функции wmdistance
if not isinstance(model, KeyedVectors):
    model = model.wv

# Функция для вычисления WMD
def calculate_wmd(text1, text2, model):
    text1 = text1.lower().split()
    text2 = text2.lower().split()
    return model.wmdistance(text1, text2)

# Вычисление WMD для каждой пары и добавление в DataFrame
best_match_df['WMD_Similarity'] = best_match_df.apply(lambda row: calculate_wmd(row['Текст требования'], row['Текст параграфа'], model), axis=1)

# Вывод первых нескольких строк для проверки
best_match_df.head(16)


#Интерпретация значений WMD
* Большие значения WMD: Высокое значение WMD указывает на то, что тексты сильно различаются с семантической точки зрения. Слова в этих текстах находятся далеко друг от друга в векторном пространстве, что свидетельствует о большой разнице в содержании или контексте.

* Маленькие значения WMD: Низкое значение WMD означает, что тексты семантически близки. Слова из одного текста находятся близко к словам другого текста в векторном пространстве, что указывает на схожесть или релевантность тем или идей между текстами.

* Нулевое значение WMD: Если WMD равно нулю, это означает, что тексты идентичны с точки зрения используемых слов и их векторных представлений, хотя на практике такое встречается редко, если только тексты не являются полными дубликатами.